In [1]:
# Monitor SC

In [2]:
import logging
import uuid

import smartpynector as sp
from utils import *

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("MonitorSC")

In [4]:
# Constants
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "ex": "http://example.org/",
}

MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""

H_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id .
                        ?meas ex:measuredAfter ?startTimestamp .
                        ?meas ex:measuredBefore ?endTimestamp ."""

H_MEAS_II_GRAPH_PATTERN = """?timeseries rdf:type ex:Timeseries .
                            ?timeseries ex:hasMeasurement ?meas .
                            ?timeseries ex:measuredAfter ?startTimestamp .
                            ?timeseries ex:measuredBefore ?endTimestamp .
                            ?meas rdf:type saref:Measurement .
                            ?meas saref:hasValue ?temp .
                            ?meas saref:hasTimestamp ?timestamp .
                            """

In [5]:
def start_monitor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    global ask_measurements_ki
    ask_measurements_ki = register_ask_knowledge_interaction(
        H_MEAS_II_GRAPH_PATTERN,
        "ask-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

In [6]:
start_monitor_kb(
    "http://example.org/monitor",
    "Monitor",
    "Monitor for storage",
    "http://knowledge_engine:8280/rest/",
)

2023-06-21 19:51:40 INFO registered Monitor
2023-06-21 19:51:40 INFO received issued knowledge interaction id: http://example.org/monitor/interaction/ask-measurements


In [7]:
# java.util.concurrent.CompletionException: java.lang.IllegalArgumentException: KB gave outgoing binding Binding [map={room_id="http://0.0.0.0:8001/thermostat/rooms/1", temp="13", device_id="http://0.0.0.0:8001/thermostat/devices/1", meas="http://0.0.0.0:8001/thermostat/measurements/07ade4c8-8221-4679-9348-0972cae20ad8", endTimestamp="2023-06-21T15:28:09+00:00", startTimestamp="2023-06-20T21:00:00+00:00", timestamp="2023-06-20T21:56:34+00:00"}], but this doesn't have a matching incoming binding!

In [8]:
def graph_data(start_time, end_time):
    global histhistorical_measurements
    historical_measurements = ask(
        [
            {
                "timeseries": f"<{THERMOSTAT_API_URL}/timeseries/{str(uuid.uuid4())}>",
                "startTimestamp": f'"{start_time}"',
                "endTimestamp": f'"{end_time}"',
            }
        ],
        ask_measurements_ki,
        "http://example.org/monitor",
        "http://knowledge_engine:8280/rest/",
    )

    logger.info(f"Historical measurements: {historical_measurements}")

In [9]:
from datetime import datetime

from IPython.display import display
from ipywidgets import Button, DatetimePicker, HBox


def convert_datetimes(start_widget, end_widget):
    start_datetime_obj = (
        start_widget.value.astimezone().replace(microsecond=0).isoformat()
    )  # ISO 8601
    end_datetime_obj = (
        end_widget.value.astimezone().replace(microsecond=0).isoformat()
    )  # ISO 8601
    return start_datetime_obj, end_datetime_obj


start_widget = DatetimePicker(description="Start Time:", disabled=False)
start_widget.value = datetime.now().astimezone().replace(microsecond=0)

end_widget = DatetimePicker(description="End Time:", disabled=False)
end_widget.value = datetime.now().astimezone().replace(microsecond=0)

submit_button = Button(description="Submit", button_style="danger")

display(HBox([start_widget, end_widget, submit_button]))


def on_button_click(button):
    start_timestamp, end_timestamp = convert_datetimes(start_widget, end_widget)
    logger.info(f"Start: {start_timestamp}, End: {end_timestamp}")
    graph_data(start_timestamp, end_timestamp)


submit_button.on_click(on_button_click)